### Basic introduction

#### Adjacency Matrix

In [5]:
adjacency_matrix = [[0,1,0,1],[1,0,1,1],[0,1,0,0],[1,1,0,0]]
adjacency_matrix_directed = [[0,1,0,1],[0,0,1,0],[0,0,0,1],[0,0,0,0]]

Some terms used in the textbook

**measure** = the number of vertices in the graph

**size** = the number of edges in the graph

#### Some basic statistics

In [7]:
num_species=len(adjacency_matrix_directed[0])
num_species

4

In [9]:
# number of links or predations (or edges) in the non zero elements
# of the adjacency matrix
num_predations = 0
for i in range(num_species):
    for j in range(num_species):
        if adjacency_matrix_directed[i][j]!=0:
            num_predations+=1

num_predations

4

In [13]:
row_sums=[0,0,0,0]
column_sums=[0,0,0,0]
for i in range(num_species):
    for j in range(num_species):
        row_sums[i]+=adjacency_matrix_directed[i][j]
        column_sums[j]+=adjacency_matrix_directed[i][j]

Classes of species:

Basal:  Only predators

Top:  Only prey

Intermediate:  Both

In [20]:
basal=0
top=0
intermediate=0
for i in range(num_species):
    if row_sums[i]==0 and column_sums[i]>0:
        basal+=1
    elif row_sums[i]>0 and column_sums[i]:
        intermediate+=1
    else:
        top+=1

print("Class sizes:  \n Basal: {} \n Top: {} \n Intermediate: {}".format(basal, top, intermediate))

Class sizes:  
 Basal: 1 
 Top: 1 
 Intermediate: 2


### Degree node calculations using networkX

In [21]:
import networkx as nx

In [22]:
# generate graph
G=nx.Graph()
for i in [1,2,3,4]:
    G.add_node(i)

In [24]:
G.add_edge(1,2)
G.add_edge(1,4)
G.add_edge(2,3)
G.add_edge(2,4)

In [25]:
G.degree(2)

3